In [26]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
print("Python Version:", sys.version)

Python Version: 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [2]:
# df1=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170916.txt")
# df2=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170909.txt")
# df3=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170902.txt")
# df4=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170819.txt")
# df5=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170812.txt")


In [3]:
# df6=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170805.txt")
# df7=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170729.txt")
# df8=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170722.txt")
# df9=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170715.txt")
# df10=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170708.txt")


In [4]:
# df11=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170701.txt")
# df12=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt")
# df13=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170617.txt")
# df14=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170610.txt")
# df15=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170603.txt")
# df16=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170527.txt")
# df17=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170520.txt")


In [5]:
# df18=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt")
# df19=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170506.txt")
# df20=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170429.txt")
# df21=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170422.txt")
# df22=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt")
# df23=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170408.txt")
# df24=pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_170401.txt")

In [6]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24])

In [3]:
#df.to_csv("/home/sufyan/Metis/Week1/AllData.csv")
df=pd.read_csv("/home/sufyan/Metis/Week1/AllData.csv")

In [4]:
#converts datetime strings to pd datetime objects
df["DATETIME"]=df["DATE"]+" "+df["TIME"]
df["DATETIME"]=pd.to_datetime(df["DATETIME"], format="%m/%d/%Y %H:%M:%S")

In [5]:
#Calculates NetEntries
df["NETENTRIES"]=df["ENTRIES"]
df.NETENTRIES=df['ENTRIES']-df['NETENTRIES'].shift(1)
#removes incorrect values where the turnstile has changed
dfmask= df.SCP != df.SCP.shift(1)
df.NETENTRIES[dfmask]=np.nan

/home/sufyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
#removes outliers
df=df[(df.NETENTRIES<100000) & (df.NETENTRIES>0)]

In [7]:
#imports Station Entrance GPS coords
dfX=pd.read_csv("http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv")
dfX = dfX[['Station_Name','Station_Longitude','Station_Latitude']].copy()
dfX.drop_duplicates().reset_index();


In [8]:
#Finds closest matches of station names between dfX (GPS) and df (data)
from difflib import get_close_matches
AllStations=list(dfX.Station_Name.unique())
OurStations=list(df.STATION.unique())
SameNames=[]
for name in AllStations:
    TempLen=get_close_matches(name, OurStations, cutoff=.5)
    if len(TempLen) >1:
        SameNames.append((name, TempLen[0]))

In [9]:
SameNames;

In [10]:
#removes incorrect matches found in SameNames
mylist=[('62nd St', '96 ST'), ('34th St', '53 ST'), ('138th St', '18 ST'),('Av P', 'BAY PKWY'), ('95th St', '96 ST'), ('8th St', '88 ST'),('9th St', '96 ST'),('5th Av', '75 AV')]
for i in mylist:
    SameNames.remove(i)


In [15]:
#Creates a Dict of Names
FirstTry=list(SameNames)
len(FirstTry)
FirstDict={}
for a,b in FirstTry:
    FirstDict[a]=b

In [31]:
NewSNames=pd.read_csv("/home/sufyan/Metis/Week1/sf17_ds8/projects/01-benson/StationNames.csv")
for i in range(0,len(NewSNames)):
    FirstTry.append((NewSNames.Old[i],NewSNames.New[i]))
for a,b in FirstTry:
    FirstDict[a]=b

In [14]:
FirstTry.append(('Canal St','CANAL ST'))
FirstTry.append(('Canal St (UL)','CANAL ST'))
FirstTry.append(('Fulton St','FULTON ST'))
FirstTry.append(('Times Square-42nd St','TIMES SQ-42 ST'))
FirstTry.append(('42nd St','42 ST-PORT AUTH'))
FirstTry.append(('14th St-Union Square', '14 ST-UNION SQ'))
FirstTry.append(('Grand Central-42nd St','GRD CNTRL-42 ST'))
FirstTry.append(('Pennsylvania Av', '34 ST-PENN STA'))
# FirstDict['']=''
# FirstDict['']=''
# FirstDict['']=''
# FirstDict['']=''
# FirstDict['']=''
# FirstDict['']=''
# FirstDict['']=''
#didnt include 34 Herald, 34 Hudson, 34 Penn. Not able to identify stations

In [39]:
#dfX.to_csv("/home/sufyan/Metis/Week1/SecondIndex.csv")
list(dfX.Station_Name.unique())
#FirstTry

['Ditmars Blvd',
 'Astoria Blvd-Hoyt Av',
 '30 Av-Grand Av',
 'Broadway',
 '36 Av-Washington Av',
 '39 Av-Beebe Av',
 '5th Av',
 '57th St',
 '49th St',
 '28th St',
 '23rd St',
 'Union Square',
 '8th St',
 'Prince St',
 'City Hall',
 'Cortlandt St',
 'Rector St',
 'Whitehall St',
 'Lawrence St',
 'DeKalb Av',
 'Atlantic Av-Barclays Ctr',
 'Pacific St',
 'Union St',
 '9th St',
 'Prospect Av',
 '25th St',
 '36th St',
 '45th St',
 '53rd St',
 '59th St',
 'Bay Ridge Av',
 '77th St',
 '86th St',
 '95th St',
 '7th Av',
 'Prospect Park',
 'Parkside Av',
 'Church Av',
 'Beverly Rd',
 'Cortelyou Rd',
 'Newkirk Av',
 'Av H',
 'Av J',
 'Av M',
 'Kings Highway',
 'Av U',
 'Neck Rd',
 'Sheepshead Bay',
 'Brighton Beach',
 'Ocean Parkway',
 'West 8th St',
 'Stillwell Av',
 '9th Av',
 'Fort Hamilton Parkway',
 '50th St',
 '55th St',
 '71st St',
 '79th St',
 '18th Av',
 '20th Av',
 'Bay Parkway',
 '25th Av',
 'Bay 50th St',
 '8th Av',
 'New Utrecht Av',
 'Bay Parkway-22nd Av',
 '121st St',
 '111th St',

In [32]:
dfX.Station_Name.unique()
dfX2 = dfX[['Station_Name','Station_Longitude','Station_Latitude']].copy()
dfX2=dfX.drop_duplicates().reset_index()


In [40]:
#replaces station names in first dict
dfX2=dfX2.replace(FirstDict, regex=True)
set(dfX2.Station_Name)

{'1 AV',
 '103 ST',
 '104th St-102nd St',
 '104th St-Oxford Av',
 '110 ST',
 '110 ST-Central Park North',
 '111 ST',
 '111 ST-Greenwood Av',
 '116 ST',
 '116 ST-COLUMBIA',
 '121 ST',
 '125 ST',
 '135 ST',
 '137 ST CITY COL',
 '138/GRAND CONC',
 '14 ST',
 '14 ST-UNION SQ',
 '145 ST',
 '149 ST-3 AV',
 '149/GRAND CONC',
 '155 ST',
 '157 ST',
 '163 ST-AMSTERDM',
 '167 ST',
 '168 ST',
 '168 ST - Washington Heights',
 '169 ST',
 '170 ST',
 '174 ST',
 '174-175 STs',
 '175 ST',
 '176 ST',
 '18 AV',
 '18 ST',
 '181 ST',
 '182-183 STS',
 '183 ST',
 '190 ST',
 '191 ST',
 '2 AV',
 '20 AV',
 '207 ST',
 '21 ST',
 '215 ST',
 '219 ST',
 '225 ST',
 '23 ST',
 '23 ST-Ely Av',
 '231 ST',
 '233 ST',
 '238 ST',
 '238 ST-Nereid Av',
 '25 AV',
 '25 ST',
 '28 ST',
 '3 AV',
 '3 AV 138 ST',
 '30 Av-Grand Av',
 '33 ST',
 '33 ST-RAWSON ST',
 '34th St',
 '36 Av-Washington Av',
 '36 ST',
 '39 AV',
 '40 ST LOWERY ST',
 '45 Rd-Court House Sq',
 '45 ST',
 '46 ST',
 '47-50 STs Rockefeller Center',
 '49 ST',
 '4th Av',
 

In [21]:
FirstTryList=[]
for a,b in FirstTry:
    FirstTryList.append(b)
FirstTry

['57 ST',
 '49 ST',
 '28 ST',
 '23 ST',
 '25 ST',
 '36 ST',
 '45 ST',
 '53 ST',
 '59 ST',
 '77 ST',
 '86 ST',
 '7 AV',
 '9 AV',
 '50 ST',
 '55 ST',
 '71 ST',
 '79 ST',
 '18 AV',
 '20 AV',
 '25 AV',
 'BAY 50 ST',
 '8 AV',
 '121 ST',
 '111 ST',
 '3 AV',
 '1 AV',
 'EAST 105 ST',
 '190 ST',
 '181 ST',
 '175 ST',
 '155 ST',
 '145 ST',
 '135 ST',
 '125 ST',
 '116 ST',
 '103 ST',
 '96 ST',
 '72 ST',
 '72 ST',
 '14 ST',
 '174-175 STS',
 '170 ST',
 '167 ST',
 '21 ST',
 '2 AV',
 '169 ST',
 '75 AV',
 '67 AV',
 '65 ST',
 '46 ST',
 '5 AV/53 ST',
 '238 ST',
 '231 ST',
 '215 ST',
 '191 ST',
 '157 ST',
 '18 ST',
 'E 149 ST',
 '183 ST',
 '176 ST',
 '110 ST',
 '51 ST',
 '33 ST',
 '233 ST',
 '225 ST',
 '219 ST',
 'E 180 ST',
 '174 ST',
 '6 AV',
 '168 ST']

In [41]:
FirstTry

[('57th St', '57 ST'),
 ('49th St', '49 ST'),
 ('28th St', '28 ST'),
 ('23rd St', '23 ST'),
 ('25th St', '25 ST'),
 ('36th St', '36 ST'),
 ('45th St', '45 ST'),
 ('53rd St', '53 ST'),
 ('59th St', '59 ST'),
 ('77th St', '77 ST'),
 ('86th St', '86 ST'),
 ('7th Av', '7 AV'),
 ('9th Av', '9 AV'),
 ('50th St', '50 ST'),
 ('55th St', '55 ST'),
 ('71st St', '71 ST'),
 ('79th St', '79 ST'),
 ('18th Av', '18 AV'),
 ('20th Av', '20 AV'),
 ('25th Av', '25 AV'),
 ('Bay 50th St', 'BAY 50 ST'),
 ('8th Av', '8 AV'),
 ('121st St', '121 ST'),
 ('111th St', '111 ST'),
 ('3rd Av', '3 AV'),
 ('1st Av', '1 AV'),
 ('East 105th St', 'EAST 105 ST'),
 ('190th St', '190 ST'),
 ('181st St', '181 ST'),
 ('175th St', '175 ST'),
 ('155th St', '155 ST'),
 ('145th St', '145 ST'),
 ('135th St', '135 ST'),
 ('125th St', '125 ST'),
 ('116th St', '116 ST'),
 ('103rd St', '103 ST'),
 ('96th St', '96 ST'),
 ('72nd St', '72 ST'),
 ('42nd St', '72 ST'),
 ('14th St', '14 ST'),
 ('174-175th Sts', '174-175 STS'),
 ('170th St',

In [278]:
#Probably Useless...didnt use dfMain Delete???
# dfMain = pd.DataFrame([])
# %time
# for name in FirstTryList:
#     dfMain=pd.concat([dfMain, df[df['STATION'].str.match(name)]])


In [160]:
#creates a dataframe with only matching stations 
#delete????
dfX3 = pd.DataFrame([])
for name in FirstTryList:
    dfX3=pd.concat([dfX3, df[df['STATION'].str.match(name)]])

In [161]:
#dfMain.to_csv("/home/sufyan/Metis/Week1/df[Index].csv")

<h2>Starts Hourly Processing</h2>

In [35]:
dfhourly=pd.read_csv("/home/sufyan/Metis/Week1/Station_Hourly.csv")
dfX2=dfX2.drop_duplicates(['Station_Name'])

In [36]:
merged_inner = pd.merge(left=df,right=dfX2, left_on='STATION', right_on='Station_Name')
merged_hourly=pd.merge(left=dfhourly, right=dfX2,left_on='Stations',right_on='Station_Name')
merged_hourly.drop_duplicates().reset_index()

,level_0,Unnamed: 0,Stations,0.0,0.1,0.2,0.3,0.4,0.5,0.6,...,23.4,23.5,23.6,23.7,23.8,23.9,index,Station_Name,Station_Longitude,Station_Latitude
0,0,0,28 ST,69.00,67.100,65.20,63.300,61.40,59.500,57.60,...,91.40,87.666667,83.933333,80.200,76.466667,72.733333,30,28 ST,-73.988691,40.745494
1,1,1,25 AV,45.50,43.850,42.20,40.550,38.90,37.250,35.60,...,55.40,53.750000,52.100000,50.450,48.800000,47.150000,149,25 AV,-73.986829,40.597704
2,2,2,225 ST,51.50,50.150,48.80,47.450,46.10,44.750,43.40,...,59.60,58.250000,56.900000,55.550,54.200000,52.850000,1007,225 ST,-73.860341,40.888022
3,3,3,125 ST,102.00,98.900,95.80,92.700,89.60,86.500,83.40,...,121.40,118.166667,114.933333,111.700,108.466667,105.233333,301,125 ST,-73.952343,40.811109
4,4,4,8 AV,162.25,157.325,152.40,147.475,142.55,137.625,132.70,...,191.80,186.875000,181.950000,177.025,172.100000,167.175000,157,8 AV,-74.011719,40.635064
5,5,5,215 ST,17.00,16.650,16.30,15.950,15.60,15.250,14.90,...,21.05,20.375000,19.700000,19.025,18.350000,17.675000,723,215 ST,-73.915279,40.869444
6,6,6,46 ST,31.50,30.450,29.40,28.350,27.30,26.250,25.20,...,37.80,36.750000,35.700000,34.650,33.600000,32.550000,647,46 ST,-73.913333,40.756312
7,7,7,183 ST,63.00,61.900,60.80,59.700,58.60,57.500,56.40,...,72.60,71.000000,69.400000,67.800,66.200000,64.600000,905,183 ST,-73.903879,40.858407
8,8,8,1 AV,154.00,150.775,147.55,144.325,141.10,137.875,134.65,...,175.15,171.625000,168.100000,164.575,161.050000,157.525000,231,1 AV,-73.981628,40.730953
9,9,9,21 ST,40.75,39.275,37.80,36.325,34.85,33.375,31.90,...,49.60,48.125000,46.650000,45.175,43.700000,42.225000,497,21 ST,-73.942836,40.754203


In [403]:
set(merged_inner.STATION);

In [37]:
merged_inner.to_csv("/home/sufyan/Metis/Week1/merged[Index]wGPS2.csv")

In [181]:
len(merged_inner.STATION.unique())

65

In [182]:
len(FirstTryList)##5 missing values from final list...why?

70

<h2>MAPPING</h2>

In [237]:
import geopandas as gp#must run pip install geopandas
import fiona
import folium #pip install folium


#shp = gp.GeoDataFrame.from_file('/home/sufyan/Metis/Week1/sf17_ds8/projects/01-benson/subway_entrances_may2016.shp')
shp=gp.read_file('/home/sufyan/Metis/Week1/sf17_ds8/projects/01-benson/subway_entrances_may2016.shp')


In [289]:
#replaces Station names in shp to match those in df
shpSTN=shp.replace(FirstDict, regex=True)

,Division,Line,Station_Na,Station_La,Station_Lo,Route_1,Route_2,Route_3,Route_4,Route_5,...,ADA_Notes,Free_Cross,North_Sout,East_West_,Corner,Latitude,Longitude,GEOID,NAMELSAD,geometry
0,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,...,,TRUE,31st St,23 AVe,NW,40.775149,-73.912074,36081,Queens County,POINT (1008602.708067201 221696.7163773818)
1,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,...,,TRUE,31st St,23 AVe,NE,40.774810,-73.912151,36081,Queens County,POINT (1008581.505385144 221573.1859671536)
2,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,...,,TRUE,31st St,23 AVe,NE,40.775025,-73.911891,36081,Queens County,POINT (1008653.438676419 221651.5899670543)
3,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,,,,...,,TRUE,31st St,23 AVe,NW,40.774938,-73.912337,36081,Queens County,POINT (1008529.942380719 221619.7690283604)
4,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,...,,TRUE,31st St,Hoyt Ave South,SW,40.770313,-73.917978,36081,Queens County,POINT (1006969.137944508 219933.2144234618)
5,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,...,,TRUE,31st St,Hoyt Ave South,SE,40.770170,-73.917755,36081,Queens County,POINT (1007030.955243417 219881.172700884)
6,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,...,,TRUE,31st St,Hoyt Ave North,NW,40.771009,-73.917167,36081,Queens County,POINT (1007193.535640308 220187.0016797123)
7,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,,,,...,,TRUE,31st St,Hoyt Ave North,NE,40.770870,-73.916935,36081,Queens County,POINT (1007257.844267205 220136.420219846)
8,BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,...,,TRUE,31st St,30th Ave,SW,40.766787,-73.921629,36081,Queens County,POINT (1005959.00311036 218647.6502172686)
9,BMT,Astoria,30 Av-Grand Av,40.766779,-73.921479,N,Q,,,,...,,TRUE,31st St,30th Ave,SE,40.766700,-73.921406,36081,Queens County,POINT (1006020.803204809 218616.0085940187)


In [292]:
iteration=0

In [335]:
#makes the dot overlay
#code not used in final project

from bokeh.plotting import figure,save
from bokeh.models import ColumnDataSource
import bokeh.plotting
from shapely.geometry import Point
from geopandas import GeoDataFrame

x=500
y=500
Size=10

# psource=ColumnDataSource(shpSTN)
# iteration+=1
# dots=figure(title="Dots"+str(iteration))
# dots.circle('x','y',source=psource, color='red', size=Size)
# plt.show()

# dots = shpSTN[['Longitude', 'Latitude']].apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
# geo_banks = gp.GeoDataFrame({"geometry": dots, "Station_Na": shpSTN["Station_Na"]})

# for each in shpSTN[0:MAX_RECORDS].iterrows():
#     map.simple_marker(
#         location = [each[1]['Y'],each[1]['X']],
#         clustered_marker = True)
# display(map)

# geometry = [Point(xy) for xy in zip(shpSTN.Longitude, shpSTN.Latitude)]
# #df = df.drop(['Lon', 'Lat'], axis=1)
# crs = {'init': 'epsg:4326'}
# geo_df = GeoDataFrame(shpSTN, crs=crs, geometry=geometry)
# geo_df

# for i in range(0,len(shpSTN)):
#     locations = shpSTN[['Latitude', 'Longitude']]
# locationlist = locations.values.tolist()
# len(locationlist)
# locationlist
# maprough = folium.Map(location=[40.775036, -73.912034], zoom_start=12)
# for point in range(0, len(locationlist)):
#     folium.CircleMarker(locationlist[point], popup=shpSTN['Station_Na'][point]).add_to(maprough)
# maprough
map_2 = folium.Map(location=[40.775036, -73.912034], zoom_start=12)
folium.CircleMarker([40.775036, -73.912034],radius=50.5,popup='Laurelhurst Park',color='#3186cc',fill_color='#3186cc',fill=True, fill_opacity= 1).add_to(map_2)
map_2

In [ ]:
#code not used in final project
map_2 = folium.Map(location=[40.775036, -73.912034], zoom_start=12)
folium.CircleMarker([40.775036, -73.912034],radius=50.5,popup='Laurelhurst Park',color='#3186cc',fill_color='#3186cc',fill=True, fill_opacity= 1).add_to(map_2)
map_2

In [418]:
#Creates maps for every 6 seconds 
R=0
lat=0
long=0
myflt=0.0
fileint=0
map_final=folium.Map(location=[40.775036, -73.912034], zoom_start=11.25)
for myint in range(0,240):
    for i in range(0,len(merged_hourly)):    
        R=merged_hourly[str(myint/10)][i]*.15
        lat=merged_hourly['Station_Latitude'][i]
        long=merged_hourly['Station_Longitude'][i]
        stname=merged_hourly['Station_Name'][i]
        folium.CircleMarker([lat, long],radius=R,popup=stname,color='#3186cc',fill_color='#3186cc',fill=True, fill_opacity= .7).add_to(map_final)
        myflt+=+.1
    map_final.save("/home/sufyan/Metis/Week1/ImageSeries/"+str(fileint)+'.html')
    map_final=5
    map_final=folium.Map(location=[40.775036, -73.912034], zoom_start=11.25)
    fileint+=1